In [1]:
import pandas as pd

In [2]:
#reading datasets
vehiculos = pd.read_csv('BD_VEHICULOS.CSV')
cobranzas = pd.read_csv('BD_COBRANZAS.CSV')
clientes = pd.read_csv('BD_CLIENTES.CSV')
interac_veh = pd.read_csv('BD_INTERAC_VEH.CSV')
interac_noveh = pd.read_csv('BD_INTERAC_NOVEH.csv')

In [3]:
#cleaning datasets
vehiculos = vehiculos.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
cobranzas = cobranzas.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
clientes = clientes.drop(['Unnamed: 0'], axis=1)
interac_veh = interac_veh.drop(['Unnamed: 0'], axis=1)
interac_noveh = interac_noveh.drop(['Unnamed: 0'], axis=1)

In [4]:
#Pivoting datasets "interac_veh" and "interac_noveh"
interac_veh['contador'] = 1
p_interac_veh = pd.pivot_table(interac_veh,values=['contador'],index=['PERIODO', 'CUC'],
                               columns=['TIPCASO.RN_DESCRIPTOR'], aggfunc='count', fill_value=0)
p_interac_veh = p_interac_veh.reset_index()
p_interac_veh.columns = p_interac_veh.columns.droplevel()
p_interac_veh.columns = ['PERIODO','CUC', 'Atenciones', 'Consulta', 'Disconformidades', 'Emergencia',
                         'Fidelizacion', 'Operacion', 'Operaciones', 'Reclamo', 'Relacionamiento',
                         'Requerimientos', 'Solicitud', 'Sugerencias']


interac_noveh['contador'] = 1
p_interac_noveh = pd.pivot_table(interac_noveh,values=['contador'],index=['PERIODO', 'CUC'],
                               columns=['TIPCASO.RN_DESCRIPTOR'], aggfunc='count', fill_value=0)
p_interac_noveh = p_interac_noveh.reset_index()
p_interac_noveh.columns = p_interac_noveh.columns.droplevel()
p_interac_noveh.columns = ['PERIODO','CUC', 'Atenciones', 'Consulta', 'Disconformidades', 'Emergencia',
                         'Fidelizacion', 'Operacion', 'Operaciones', 'Reclamo', 'Relacionamiento',
                         'Requerimientos', 'Solicitud', 'Sugerencias']

In [5]:
#merge datasets
dataset = vehiculos.merge(clientes, how='left', on='CUC')
dataset = dataset.merge(cobranzas, how='left', left_on=['CUC','P_AUTO','PERIODO'], right_on=['CUC','P_AUTO','PERIODO'])
dataset = dataset.merge(p_interac_veh, how='left', left_on=['CUC','PERIODO'], right_on=['CUC', 'PERIODO'])
dataset = dataset.merge(p_interac_noveh, how='left', left_on=['CUC','PERIODO'], right_on=['CUC', 'PERIODO'])

In [7]:
#creating "Deuda" variable and "Target" variable
dataset['Deuda'] = 0
dataset.loc[dataset['NUM_FACTURAS']>0,'Deuda'] = 1

dataset['new_Deuda'] = dataset['Deuda'] + dataset.groupby(['ID_UNICO'])['Deuda'].shift(-1)
dataset['new_Deuda'].fillna(0, inplace=True)
dataset['Target'] = 0
dataset.loc[((dataset['Deuda']==0) & (dataset['new_Deuda']==1)),'Target'] = 1

In [8]:
#export dataset
dataset.to_csv('dataset_full.csv', index=False)